# Main-code for House-Amneties-Neighbourhood-Crime-Income

In [1]:
# 3rd party imports
import os
import json
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pylab as plt
from bs4 import BeautifulSoup
#!pip install pandas openpyxl


# Configure Notebook
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set_context("notebook")
import warnings
warnings.filterwarnings('ignore')
%config Completer.use_jedi = False


In [2]:
# Load CSV 
tn_df = pd.read_csv('V_CensusLocalAreaProfiles2016.csv', encoding='Windows-1252')
tnT_df=tn_df.T

# View DataFrame
tn_df.T

,0,1,2,3,4,5,6,7,8,9,...,5583,5584,5585,5586,5587,5588,5589,5590,5591,5592
The data shown here is provided by Statistics Canada from the 2016 Census as a custom data order for the City of Vancouver using the City's 22 local planning areas,The data may be reproduced provided they are c...,NaN,CENSUS DATA FOR CITY OF VANCOUVER LOCAL AREAS...,ID,1,2,3,4,5,6,...,5484,5485,5486,5487,5488,5489,5490,5491,5492,5493
Unnamed: 1,NaN,NaN,NaN,Variable,Total - Age groups and average age of the pop...,0 to 14 years,0 to 4 years,5 to 9 years,10 to 14 years,15 to 64 years,...,None,English,French,Non-official language,Aboriginal,Non-Aboriginal,English and French,English and non-official language,French and non-official language,"English, French and non-official language"
Unnamed: 2,NaN,NaN,NaN,Arbutus-Ridge,"15,295",2015,455,685,880,9805,...,3120,135,40,355,0,360,0,10,0,0
Unnamed: 3,NaN,NaN,NaN,Downtown,"62,030",4000,2080,1105,810,51275,...,17290,360,255,1300,0,1300,0,10,15,0
Unnamed: 4,NaN,NaN,NaN,Dunbar-Southlands,"21,425",3545,675,1225,1650,14215,...,5035,100,105,335,10,335,0,0,10,0
Unnamed: 5,NaN,NaN,NaN,Fairview,"33,620",2580,1240,760,580,25140,...,11900,95,255,500,0,505,0,0,20,0
Unnamed: 6,NaN,NaN,NaN,Grandview-Woodland,"29,175",3210,1320,1025,865,22535,...,9525,150,180,320,15,305,0,0,10,0
Unnamed: 7,NaN,NaN,NaN,Hastings-Sunrise,"34,575",4595,1510,1560,1525,23945,...,8805,230,65,750,0,750,0,15,0,0
Unnamed: 8,NaN,NaN,NaN,Kensington-Cedar Cottage,"49,325",7060,2515,2390,2160,35385,...,13525,360,150,1125,0,1125,0,20,20,0
Unnamed: 9,NaN,NaN,NaN,Kerrisdale,"13,975",1880,430,600,845,9395,...,2900,75,25,360,0,360,0,10,0,0


## Filtering and Cleaning the dataframe to obtain the required data set.

In [3]:
# Extract the first data row (index 1, which is the second row of the sheet)
first_row = tn_df.T.iloc[1].astype(str)

# Keywords to search for
keywords = [
    'Income statistics in 2020 for the population aged 15 years and over in private households - 25% sample data',
   # 'Income', 
    'tax',
    'after-tax income',
    'after-tax income in 2020',
    'Median total income'
]

# Initialize a dictionary to store all indexes for each keyword
all_keyword_indexes = {}

# Find keywords in the first row (case-insensitive matching) and their corresponding indexes
for keyword in keywords:
    # Find all indexes for the keyword
    indexes = first_row[first_row.str.contains(keyword, case=False, na=False)].index.tolist()
    
    # If keyword is found, store the indexes
    if indexes:
        # Store the indexes for the keyword
        all_keyword_indexes[keyword] = indexes
        
        # Calculate and print only the min and max indexes
        min_index = min(indexes)  # Get the minimum index
        max_index = max(indexes)  # Get the maximum index
        print(f"Keyword '{keyword}' found at min index: {min_index} and max index: {max_index}")

# At this point, the full list of indexes is saved in the all_keyword_indexes dictionary.
# To check saved indexes for a specific keyword (if needed later):
# print(all_keyword_indexes)


Keyword 'tax' found at min index: 1887 and max index: 4178
Keyword 'after-tax income' found at min index: 1887 and max index: 4178
Keyword 'Median total income' found at min index: 1886 and max index: 4176


In [4]:
# Access the actual values at columns 60 and 1949 (first row)
value_at_index_60 = tn_df.T.iloc[0,92]  # 61st column, first row (index 0)
value_at_index_1949 = tn_df.T.iloc[0, 424]  # 1950th column, first row (index 0)

# Optionally, get the column names at those indices
column_name_60 = tn_df.T.columns[92]  # 61st column name
column_name_1949 = tn_df.T.columns[424]  # 1950th column name

# Print the results
print(f"Value at column {column_name_60} (index 60): {value_at_index_60}")
print(f"Value at column {column_name_1949} (index 1949): {value_at_index_1949}")


Value at column 92 (index 60): 88
Value at column 424 (index 1949): 404


In [5]:
# Step 1: Extract the first column (assuming it contains labels or identifiers)
first_column = tn_df.T.iloc[:, 3]

# Step 2: Filter the columns based on index range (between 60 and 168)
filtered_columns = tn_df.T.iloc[:, 1883:2018]  # Select columns from index 60 to 168 (inclusive of 60, exclusive of 169)

# Step 3: Create a new DataFrame with the first column and the filtered columns
filtered_df = pd.concat([first_column, filtered_columns], axis=1)

# Step 4: Print the resulting filtered DataFrame
(filtered_df)


,3,1883,1884,1885,1886,1887,1888,1889,1890,1891,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
The data shown here is provided by Statistics Canada from the 2016 Census as a custom data order for the City of Vancouver using the City's 22 local planning areas,ID,1856,1857,1858,1859,1860,1861,1862,1863,1864,...,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986
Unnamed: 1,Variable,Total - Income statistics in 2015 for the popu...,Number of total income recipients aged 15 ye...,Average total income in 2015 among recipie...,Median total income in 2015 among recipien...,Number of after-tax income recipients aged 1...,Average after-tax income in 2015 among rec...,Median after-tax income in 2015 among reci...,Number of market income recipients aged 15 y...,Average market income in 2015 among recipi...,...,"$30,000 to $39,999","$40,000 to $49,999","$50,000 to $59,999","$60,000 to $69,999","$70,000 to $79,999","$80,000 and over","$80,000 to $89,999","$90,000 to $99,999","$100,000 and over",Percentage with after-tax income
Unnamed: 2,Arbutus-Ridge,13055,12185,62675,30929,12180,48943,27946,11300,62995,...,490,390,325,320,155,1040,140,185,720,92.7
Unnamed: 3,Downtown,54905,53630,63251,41858,53720,49390,36918,47865,67870,...,2515,2700,2440,2075,1620,5750,1320,1050,3375,98.6
Unnamed: 4,Dunbar-Southlands,17785,16685,78117,40463,16715,59021,35459,15640,79592,...,640,585,470,455,300,2285,340,315,1635,93.3
Unnamed: 5,Fairview,30130,29655,61627,46940,29685,49734,40815,28220,61142,...,1470,1575,1430,1105,910,2490,645,365,1480,98.8
Unnamed: 6,Grandview-Woodland,25915,25435,42896,32438,25465,36036,28951,22410,44009,...,1625,1290,950,685,490,1090,365,220,515,98.2
Unnamed: 7,Hastings-Sunrise,29410,28420,38258,27255,28440,32460,25163,24865,38305,...,1755,1390,1080,690,520,985,320,165,495,97.1
Unnamed: 8,Kensington-Cedar Cottage,41795,40440,38411,28356,40485,32767,26205,36025,38437,...,2475,2165,1445,905,755,1535,555,265,715,96.9
Unnamed: 9,Kerrisdale,12060,11435,77248,35064,11450,57131,31259,10570,79395,...,405,395,360,240,185,1235,175,170,885,94.8


In [6]:
# Step 1: Reset the row index and add it as a column (this step you already have)
filtered_df_reset = filtered_df.reset_index()

# Step 2: Set the row index column to the first column (this you already did)
filtered_df_reset.set_index(filtered_df_reset.columns[0], inplace=True)

# Step 3: Reset column headers to default (0, 1, 2, ...)
# This step will reset the column headers (the numeric indices you mentioned)
filtered_df_reset.columns = range(filtered_df_reset.shape[1])

# Confirmation message
print("Row index has been moved to the first column, and column headers have been reset.")
filtered_df_reset

Row index has been moved to the first column, and column headers have been reset.


,0,1,2,3,4,5,6,7,8,9,...,126,127,128,129,130,131,132,133,134,135
index,,,,,,,,,,,,,,,,,,,,,
The data shown here is provided by Statistics Canada from the 2016 Census as a custom data order for the City of Vancouver using the City's 22 local planning areas,ID,1856,1857,1858,1859,1860,1861,1862,1863,1864,...,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986
Unnamed: 1,Variable,Total - Income statistics in 2015 for the popu...,Number of total income recipients aged 15 ye...,Average total income in 2015 among recipie...,Median total income in 2015 among recipien...,Number of after-tax income recipients aged 1...,Average after-tax income in 2015 among rec...,Median after-tax income in 2015 among reci...,Number of market income recipients aged 15 y...,Average market income in 2015 among recipi...,...,"$30,000 to $39,999","$40,000 to $49,999","$50,000 to $59,999","$60,000 to $69,999","$70,000 to $79,999","$80,000 and over","$80,000 to $89,999","$90,000 to $99,999","$100,000 and over",Percentage with after-tax income
Unnamed: 2,Arbutus-Ridge,13055,12185,62675,30929,12180,48943,27946,11300,62995,...,490,390,325,320,155,1040,140,185,720,92.7
Unnamed: 3,Downtown,54905,53630,63251,41858,53720,49390,36918,47865,67870,...,2515,2700,2440,2075,1620,5750,1320,1050,3375,98.6
Unnamed: 4,Dunbar-Southlands,17785,16685,78117,40463,16715,59021,35459,15640,79592,...,640,585,470,455,300,2285,340,315,1635,93.3
Unnamed: 5,Fairview,30130,29655,61627,46940,29685,49734,40815,28220,61142,...,1470,1575,1430,1105,910,2490,645,365,1480,98.8
Unnamed: 6,Grandview-Woodland,25915,25435,42896,32438,25465,36036,28951,22410,44009,...,1625,1290,950,685,490,1090,365,220,515,98.2
Unnamed: 7,Hastings-Sunrise,29410,28420,38258,27255,28440,32460,25163,24865,38305,...,1755,1390,1080,690,520,985,320,165,495,97.1
Unnamed: 8,Kensington-Cedar Cottage,41795,40440,38411,28356,40485,32767,26205,36025,38437,...,2475,2165,1445,905,755,1535,555,265,715,96.9


In [7]:
# Step 1: Reset the index, which makes the current index a column
filtered_df_reset1 = filtered_df_reset.reset_index()

# # Drop the first column by index
filtered_df_reset1 = filtered_df_reset1.drop(filtered_df_reset1.columns[0], axis=1)
# filtered_df_reset
filtered_df_reset1 = filtered_df_reset1.drop(filtered_df_reset1.columns[0], axis=0)

filtered_df_reset1

,0,1,2,3,4,5,6,7,8,9,...,126,127,128,129,130,131,132,133,134,135
1,Variable,Total - Income statistics in 2015 for the popu...,Number of total income recipients aged 15 ye...,Average total income in 2015 among recipie...,Median total income in 2015 among recipien...,Number of after-tax income recipients aged 1...,Average after-tax income in 2015 among rec...,Median after-tax income in 2015 among reci...,Number of market income recipients aged 15 y...,Average market income in 2015 among recipi...,...,"$30,000 to $39,999","$40,000 to $49,999","$50,000 to $59,999","$60,000 to $69,999","$70,000 to $79,999","$80,000 and over","$80,000 to $89,999","$90,000 to $99,999","$100,000 and over",Percentage with after-tax income
2,Arbutus-Ridge,13055,12185,62675,30929,12180,48943,27946,11300,62995,...,490,390,325,320,155,1040,140,185,720,92.7
3,Downtown,54905,53630,63251,41858,53720,49390,36918,47865,67870,...,2515,2700,2440,2075,1620,5750,1320,1050,3375,98.6
4,Dunbar-Southlands,17785,16685,78117,40463,16715,59021,35459,15640,79592,...,640,585,470,455,300,2285,340,315,1635,93.3
5,Fairview,30130,29655,61627,46940,29685,49734,40815,28220,61142,...,1470,1575,1430,1105,910,2490,645,365,1480,98.8
6,Grandview-Woodland,25915,25435,42896,32438,25465,36036,28951,22410,44009,...,1625,1290,950,685,490,1090,365,220,515,98.2
7,Hastings-Sunrise,29410,28420,38258,27255,28440,32460,25163,24865,38305,...,1755,1390,1080,690,520,985,320,165,495,97.1
8,Kensington-Cedar Cottage,41795,40440,38411,28356,40485,32767,26205,36025,38437,...,2475,2165,1445,905,755,1535,555,265,715,96.9
9,Kerrisdale,12060,11435,77248,35064,11450,57131,31259,10570,79395,...,405,395,360,240,185,1235,175,170,885,94.8
10,Killarney,24675,23590,39013,29259,23630,33358,26662,20775,39112,...,1310,1215,920,685,475,850,280,185,380,95.4


In [8]:
# Set the first row as the header and remove it from the data
filtered_df_reset1.columns = filtered_df_reset1.iloc[0]  # Set the first row as column names
filtered_df_reset1 = filtered_df_reset1[1:]               # Remove the first row from the DataFrame
filtered_df_reset1.reset_index(drop=True, inplace=True)  # Reset the index
filtered_df_reset1= filtered_df_reset1.rename(columns={"Variable": "Neighbourhood"})
filtered_df_reset1

1,Neighbourhood,Total - Income statistics in 2015 for the population aged 15 years and over in private households - 25% sample data,Number of total income recipients aged 15 years and over in private households - 25% sample data,Average total income in 2015 among recipients ($),Median total income in 2015 among recipients ($),Number of after-tax income recipients aged 15 years and over in private households - 25% sample data,Average after-tax income in 2015 among recipients ($),Median after-tax income in 2015 among recipients ($),Number of market income recipients aged 15 years and over in private households - 25% sample data,Average market income in 2015 among recipients ($),...,"$30,000 to $39,999","$40,000 to $49,999","$50,000 to $59,999","$60,000 to $69,999","$70,000 to $79,999","$80,000 and over","$80,000 to $89,999","$90,000 to $99,999","$100,000 and over",Percentage with after-tax income
0,Arbutus-Ridge,13055,12185,62675,30929,12180,48943,27946,11300,62995,...,490,390,325,320,155,1040,140,185,720,92.7
1,Downtown,54905,53630,63251,41858,53720,49390,36918,47865,67870,...,2515,2700,2440,2075,1620,5750,1320,1050,3375,98.6
2,Dunbar-Southlands,17785,16685,78117,40463,16715,59021,35459,15640,79592,...,640,585,470,455,300,2285,340,315,1635,93.3
3,Fairview,30130,29655,61627,46940,29685,49734,40815,28220,61142,...,1470,1575,1430,1105,910,2490,645,365,1480,98.8
4,Grandview-Woodland,25915,25435,42896,32438,25465,36036,28951,22410,44009,...,1625,1290,950,685,490,1090,365,220,515,98.2
5,Hastings-Sunrise,29410,28420,38258,27255,28440,32460,25163,24865,38305,...,1755,1390,1080,690,520,985,320,165,495,97.1
6,Kensington-Cedar Cottage,41795,40440,38411,28356,40485,32767,26205,36025,38437,...,2475,2165,1445,905,755,1535,555,265,715,96.9
7,Kerrisdale,12060,11435,77248,35064,11450,57131,31259,10570,79395,...,405,395,360,240,185,1235,175,170,885,94.8
8,Killarney,24675,23590,39013,29259,23630,33358,26662,20775,39112,...,1310,1215,920,685,475,850,280,185,380,95.4
9,Kitsilano,38450,37500,63092,44084,37550,49729,38721,35745,63015,...,1835,1630,1500,1305,980,3490,760,580,2150,97.5


## Loading the file from the previous pipeline step.

In [10]:
#second file
hos_t=pd.read_csv("Vancouver_houses_with_crime_data_1km.csv")
hos_t


,Unnamed: 0,Street address,Zip/Postal Code,Latitude,Longitude,Region,Price,Bedrooms,Bathrooms,Square Footage,...,geometry,Neighbourhood,Total_crimes (2020-2024),Average_total_crime (per_year),Total_severe_crimes (2020-2024),Average_severe_crimes (per_year),Total_moderate_crimes (2020-2024),Average_moderate_crimes (per_year),Total_minor_crimes (2020-2024),Average_minor_crimes (per_year)
0,0,1525 Coal Harbour Cay #B17,V6G3E7,49.291479,-123.128047,Vancouver,375000.0,1 bd,1 ba,484 sqft,...,POINT (-123.128047 49.291479),Downtown,49367.0,9873.0,6539.0,1308.0,3925.0,785.0,38903.0,7781.0
1,1,2106 SW Marine Dr,V6P6B5,49.214789,-123.155830,Vancouver,27500000.0,6 bds,9 ba,"10,709 sqft",...,POINT (-123.15583 49.214789),Kerrisdale,1477.0,295.0,107.0,21.0,402.0,80.0,968.0,194.0
2,2,3440 Broadway W #126,V6R4R2,49.264012,-123.180766,Vancouver,989988.0,3 bds,2 ba,"1,266 sqft",...,POINT (-123.180766 49.264012),Kitsilano,5811.0,1162.0,305.0,61.0,1029.0,206.0,4477.0,895.0
3,3,8481 Portside Ct,V5P4V4,49.207213,-123.061855,Vancouver,899900.0,3 bds,3 ba,"1,404 sqft",...,POINT (-123.061855 49.207213),Victoria-Fraserview,2680.0,536.0,260.0,52.0,421.0,84.0,1999.0,400.0
4,4,2870 SW Marine Dr,V6N3X9,49.225986,-123.170995,Vancouver,16900000.0,7 bds,11 ba,"11,271 sqft",...,POINT (-123.170995 49.225986),Kerrisdale,1477.0,295.0,107.0,21.0,402.0,80.0,968.0,194.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1214,1214,3576 E Georgia Street,NaN,49.278003,-123.026556,NaN,2198000.0,6 bd,4.0,NaN,...,POINT (-123.026556 49.278003),Hastings-Sunrise,4895.0,979.0,469.0,94.0,883.0,177.0,3543.0,709.0
1215,1215,3822 W 2nd Avenue,NaN,49.270197,-123.188209,NaN,4998000.0,5 bd,4.0,NaN,...,POINT (-123.188209 49.270197),West Point Grey,1463.0,293.0,100.0,20.0,353.0,71.0,1010.0,202.0
1216,1216,Tha 1199 Marinaside Crescent,NaN,49.273129,-123.120638,NaN,2398000.0,3 bd,3.0,NaN,...,POINT (-123.120638 49.273129),Downtown,49367.0,9873.0,6539.0,1308.0,3925.0,785.0,38903.0,7781.0
1217,1217,2984 W 31st Avenue,NaN,49.243942,-123.172383,NaN,4199800.0,9 bd,8.0,NaN,...,POINT (-123.172383 49.243942),Dunbar-Southlands,1840.0,368.0,172.0,34.0,367.0,73.0,1301.0,260.0


### Combining the data_sets

In [11]:
df=hos_t

column_name = 'Neighbourhood'

# Get unique entries in the column
unique_entries = df[column_name].dropna().unique()

# Convert to a list if needed (or you can leave it as an array)
unique_entries_list = unique_entries.tolist()

# Display the unique entries
len(unique_entries_list)

22

In [12]:
df=filtered_df_reset1

column_name = 'Neighbourhood'

# Get unique entries in the column
unique_entries = df[column_name].dropna().unique()

# Convert to a list if needed (or you can leave it as an array)
unique_entries_list = unique_entries.tolist()

# Display the unique entries
len(unique_entries_list)

24

In [13]:
# Load the two DataFrames
df1 = hos_t
df2 = filtered_df_reset1

# Specify the column with unique entries in df1 and the corresponding column in df2
unique_column_df1 = 'Neighbourhood'
matching_column_df2 = 'Neighbourhood'

# Clean the 'Neighbourhood' column in both DataFrames
#df1[unique_column_df1] = df1[unique_column_df1].str.strip().str.lower().replace({'\'': '', ',': '': ''}, regex=True)
df1[unique_column_df1] = df1[unique_column_df1].str.strip().str.lower().replace({'\'': ' ', ',': ' ', '-': ' ','`': ' '}, regex=True).str.replace(r'\s+', ' ', regex=True)

df2[matching_column_df2] = df2[matching_column_df2].str.strip().str.lower().replace({'\'': ' ', ',': ' ', '-': ' ','`': ' '}, regex=True).str.replace(r'\s+', ' ', regex=True)

df1[unique_column_df1] = df1[unique_column_df1].str.replace(r'\bst\.\b', 'st ', regex=True, case=False)
df2[matching_column_df2] = df2[matching_column_df2].str.replace(r'st\.', 'st ', regex=True, case=False)

df1[unique_column_df1] = df1[unique_column_df1].str.replace(r'\s+', ' ', regex=True)
df2[matching_column_df2] = df2[matching_column_df2].str.replace(r'\s+', ' ', regex=True)


# Step 1: Get unique entries from df1 (now cleaned)
unique_entries = df1[unique_column_df1].dropna().unique()

# Step 2: Filter rows in df2 where the matching column values are in unique_entries
filtered_df2 = df2[df2[matching_column_df2].isin(unique_entries)]

# Display the filtered dataframe
filtered_df2


1,Neighbourhood,Total - Income statistics in 2015 for the population aged 15 years and over in private households - 25% sample data,Number of total income recipients aged 15 years and over in private households - 25% sample data,Average total income in 2015 among recipients ($),Median total income in 2015 among recipients ($),Number of after-tax income recipients aged 15 years and over in private households - 25% sample data,Average after-tax income in 2015 among recipients ($),Median after-tax income in 2015 among recipients ($),Number of market income recipients aged 15 years and over in private households - 25% sample data,Average market income in 2015 among recipients ($),...,"$30,000 to $39,999","$40,000 to $49,999","$50,000 to $59,999","$60,000 to $69,999","$70,000 to $79,999","$80,000 and over","$80,000 to $89,999","$90,000 to $99,999","$100,000 and over",Percentage with after-tax income
0,arbutus ridge,13055,12185,62675,30929,12180,48943,27946,11300,62995,...,490,390,325,320,155,1040,140,185,720,92.7
1,downtown,54905,53630,63251,41858,53720,49390,36918,47865,67870,...,2515,2700,2440,2075,1620,5750,1320,1050,3375,98.6
2,dunbar southlands,17785,16685,78117,40463,16715,59021,35459,15640,79592,...,640,585,470,455,300,2285,340,315,1635,93.3
3,fairview,30130,29655,61627,46940,29685,49734,40815,28220,61142,...,1470,1575,1430,1105,910,2490,645,365,1480,98.8
4,grandview woodland,25915,25435,42896,32438,25465,36036,28951,22410,44009,...,1625,1290,950,685,490,1090,365,220,515,98.2
5,hastings sunrise,29410,28420,38258,27255,28440,32460,25163,24865,38305,...,1755,1390,1080,690,520,985,320,165,495,97.1
6,kensington cedar cottage,41795,40440,38411,28356,40485,32767,26205,36025,38437,...,2475,2165,1445,905,755,1535,555,265,715,96.9
7,kerrisdale,12060,11435,77248,35064,11450,57131,31259,10570,79395,...,405,395,360,240,185,1235,175,170,885,94.8
8,killarney,24675,23590,39013,29259,23630,33358,26662,20775,39112,...,1310,1215,920,685,475,850,280,185,380,95.4
9,kitsilano,38450,37500,63092,44084,37550,49729,38721,35745,63015,...,1835,1630,1500,1305,980,3490,760,580,2150,97.5


In [14]:
# Get entries in df1 that are not in df2
unmatched_entries = set(unique_entries) - set(df2[matching_column_df2].unique())
print("Unmatched entries:", unmatched_entries)


Unmatched entries: set()


### Getting only the subset of income dataset

In [15]:
# Select the specific row in df1 that you want to use as column headers
# For example, if it's the first row (index 0):
df1= filtered_df_reset
new_headers = df1.iloc[0]
len(new_headers)

# df2= filtered_df2
# # # Update df2's column names with these new headers
# # Keep the first column header of df2 unchanged
# df2_columns = [df2.columns[0]] + list(new_headers)

# # Update df2's columns
# df2.columns = df2_columns

# # # Confirm the change
# # print("Updated column names of df2:")
# income_1=df2


income_1 = (filtered_df2)
income_1


1,Neighbourhood,Total - Income statistics in 2015 for the population aged 15 years and over in private households - 25% sample data,Number of total income recipients aged 15 years and over in private households - 25% sample data,Average total income in 2015 among recipients ($),Median total income in 2015 among recipients ($),Number of after-tax income recipients aged 15 years and over in private households - 25% sample data,Average after-tax income in 2015 among recipients ($),Median after-tax income in 2015 among recipients ($),Number of market income recipients aged 15 years and over in private households - 25% sample data,Average market income in 2015 among recipients ($),...,"$30,000 to $39,999","$40,000 to $49,999","$50,000 to $59,999","$60,000 to $69,999","$70,000 to $79,999","$80,000 and over","$80,000 to $89,999","$90,000 to $99,999","$100,000 and over",Percentage with after-tax income
0,arbutus ridge,13055,12185,62675,30929,12180,48943,27946,11300,62995,...,490,390,325,320,155,1040,140,185,720,92.7
1,downtown,54905,53630,63251,41858,53720,49390,36918,47865,67870,...,2515,2700,2440,2075,1620,5750,1320,1050,3375,98.6
2,dunbar southlands,17785,16685,78117,40463,16715,59021,35459,15640,79592,...,640,585,470,455,300,2285,340,315,1635,93.3
3,fairview,30130,29655,61627,46940,29685,49734,40815,28220,61142,...,1470,1575,1430,1105,910,2490,645,365,1480,98.8
4,grandview woodland,25915,25435,42896,32438,25465,36036,28951,22410,44009,...,1625,1290,950,685,490,1090,365,220,515,98.2
5,hastings sunrise,29410,28420,38258,27255,28440,32460,25163,24865,38305,...,1755,1390,1080,690,520,985,320,165,495,97.1
6,kensington cedar cottage,41795,40440,38411,28356,40485,32767,26205,36025,38437,...,2475,2165,1445,905,755,1535,555,265,715,96.9
7,kerrisdale,12060,11435,77248,35064,11450,57131,31259,10570,79395,...,405,395,360,240,185,1235,175,170,885,94.8
8,killarney,24675,23590,39013,29259,23630,33358,26662,20775,39112,...,1310,1215,920,685,475,850,280,185,380,95.4
9,kitsilano,38450,37500,63092,44084,37550,49729,38721,35745,63015,...,1835,1630,1500,1305,980,3490,760,580,2150,97.5


In [16]:
# Select and convert the necessary columns to lists
columns_to_keep = ['Neighbourhood'] + ['    Average total income in 2015 among recipients ($)'] + ['    Average after-tax income in 2015 among recipients ($)'] #+ income_1.iloc[:,31:80]

# Use the columns_to_keep list to select those columns from the DataFrame
income = income_1[columns_to_keep]

# Display the new DataFrame
(income)


1,Neighbourhood,Average total income in 2015 among recipients ($),Average after-tax income in 2015 among recipients ($)
0,arbutus ridge,62675,48943
1,downtown,63251,49390
2,dunbar southlands,78117,59021
3,fairview,61627,49734
4,grandview woodland,42896,36036
5,hastings sunrise,38258,32460
6,kensington cedar cottage,38411,32767
7,kerrisdale,77248,57131
8,killarney,39013,33358
9,kitsilano,63092,49729


In [17]:
#segregating gross-after tax incomes
# Group 1: Columns 1 to N (excluding the first column)
# Adjust the range of columns as necessary
df_1 = income_1.iloc[:, 50:66]  # First half of remaining columns
df_2 = income_1.iloc[:, 103:118]   # Second half of remaining columns
#df_3 = income_1.iloc[:, 120:]   # Second half of remaining columns

# # Step 3: Add the 'Neigh' column back to each new DataFrame
df_1 = pd.concat([income,df_1], axis=1) #gross
df_2 = pd.concat([income,df_2], axis=1) #after-tax
#df_3 = pd.concat([income,df_3],axis=1)  #employement
df_2.head()

1,Neighbourhood,Average total income in 2015 among recipients ($),Average after-tax income in 2015 among recipients ($),Total - After-tax income groups in 2015 for the population aged 15 years and over in private households - 25% sample data,Without after-tax income,With after-tax income,"Under $10,000 (including loss)","$10,000 to $19,999","$20,000 to $29,999","$30,000 to $39,999","$40,000 to $49,999","$50,000 to $59,999","$60,000 to $69,999","$70,000 to $79,999","$80,000 and over","$80,000 to $89,999","$90,000 to $99,999","$100,000 and over"
0,arbutus ridge,62675,48943,13055,870,12180,2905,1910,1610,1180,960,755,660,420,1780,375,300,1100
1,downtown,63251,49390,54910,1185,53715,9205,8390,5755,5185,5315,4520,3915,2990,8445,2165,1585,4700
2,dunbar southlands,78117,59021,17790,1070,16715,3510,2310,1745,1440,1335,1085,895,720,3670,685,530,2455
3,fairview,61627,49734,30125,445,29685,3340,3695,3700,3770,3660,2990,2530,1860,4135,1175,615,2340
4,grandview woodland,42896,36036,25915,445,25470,3415,5570,4080,3570,2795,1950,1365,935,1790,605,430,755


#### Combining the income braackets to define 6 classes

In [18]:
#Gross income in classes
# Add the values of two columns (e.g., 'column1' and 'column2') and assign the result to a new column
avg_expense = 1530*12

df_1['Class 6G'] = pd.to_numeric(df_1.iloc[:,7]) + pd.to_numeric(df_1.iloc[:,8])
df_1['Class 5G'] = pd.to_numeric(df_1.iloc[:,9]) + pd.to_numeric(df_1.iloc[:,10]) +pd.to_numeric(df_1.iloc[:,11])
df_1['Class 4G'] = pd.to_numeric(df_1.iloc[:,12]) + pd.to_numeric(df_1.iloc[:,13]) 
df_1['Class 3G'] = pd.to_numeric(df_1.iloc[:,14]) + pd.to_numeric(df_1.iloc[:,15]) +pd.to_numeric(df_1.iloc[:,16])
df_1['Class 2G'] = pd.to_numeric(df_1.iloc[:,17]) 
df_1['Class 1G'] = pd.to_numeric(df_1.iloc[:,18]) 


# Print the DataFrame to confirm the new column
(df_1).head()


1,Neighbourhood,Average total income in 2015 among recipients ($),Average after-tax income in 2015 among recipients ($),Total - Total income groups in 2015 for the population aged 15 years and over in private households - 25% sample data,Without total income,With total income,"Under $10,000 (including loss)","$10,000 to $19,999","$20,000 to $29,999","$30,000 to $39,999",...,"$90,000 to $99,999","$100,000 and over","$100,000 to $149,999","$150,000 and over",Class 6G,Class 5G,Class 4G,Class 3G,Class 2G,Class 1G
0,arbutus ridge,62675,48943,13060,875,12180,2790,1820,1365,1110,...,320,1720,855,870,3185,2670,1090,2455,855,870
1,downtown,63251,49390,54905,1280,53630,8785,8050,4830,4325,...,1940,8590,5180,3405,12880,12295,6645,13025,5180,3405
2,dunbar southlands,78117,59021,17790,1100,16690,3320,2140,1615,1205,...,575,3515,1555,1965,3755,3360,1525,4725,1555,1965
3,fairview,61627,49734,30125,475,29655,3185,3495,3110,2955,...,1165,4105,2590,1520,6605,8615,4350,6895,2590,1520
4,grandview woodland,42896,36036,25915,475,25435,3300,5305,3420,2945,...,610,1745,1260,485,8725,7490,2715,3205,1260,485


In [19]:
#after tax income
# Add the values of two columns (e.g., 'column1' and 'column2') and assign the result to a new column
avg_expense = 1530

df_2['Class 6G_T'] = pd.to_numeric(df_2.iloc[:,6]) + pd.to_numeric(df_2.iloc[:,7])
df_2['Class 5G_T'] = pd.to_numeric(df_2.iloc[:,8]) + pd.to_numeric(df_2.iloc[:,9]) +pd.to_numeric(df_2.iloc[:,10])
df_2['Class 4G_T'] = pd.to_numeric(df_2.iloc[:,11]) + pd.to_numeric(df_2.iloc[:,12]) 
df_2['Class 3G_T'] = pd.to_numeric(df_2.iloc[:,13]) + pd.to_numeric(df_2.iloc[:,14]) +pd.to_numeric(df_2.iloc[:,15])
df_2['Class 2G_T'] = pd.to_numeric(df_2.iloc[:,17]) 
df_2['Class 1G_T'] = pd.to_numeric(df_2.iloc[:,17]) 

# Print the DataFrame to confirm the new column
(df_2)


1,Neighbourhood,Average total income in 2015 among recipients ($),Average after-tax income in 2015 among recipients ($),Total - After-tax income groups in 2015 for the population aged 15 years and over in private households - 25% sample data,Without after-tax income,With after-tax income,"Under $10,000 (including loss)","$10,000 to $19,999","$20,000 to $29,999","$30,000 to $39,999",...,"$80,000 and over","$80,000 to $89,999","$90,000 to $99,999","$100,000 and over",Class 6G_T,Class 5G_T,Class 4G_T,Class 3G_T,Class 2G_T,Class 1G_T
0,arbutus ridge,62675,48943,13055,870,12180,2905,1910,1610,1180,...,1780,375,300,1100,4815,3750,1415,2575,1100,1100
1,downtown,63251,49390,54910,1185,53715,9205,8390,5755,5185,...,8445,2165,1585,4700,17595,16255,8435,13600,4700,4700
2,dunbar southlands,78117,59021,17790,1070,16715,3510,2310,1745,1440,...,3670,685,530,2455,5820,4520,1980,5075,2455,2455
3,fairview,61627,49734,30125,445,29685,3340,3695,3700,3770,...,4135,1175,615,2340,7035,11130,5520,7170,2340,2340
4,grandview woodland,42896,36036,25915,445,25470,3415,5570,4080,3570,...,1790,605,430,755,8985,10445,3315,3330,755,755
5,hastings sunrise,38258,32460,29405,970,28440,4745,6585,4965,3730,...,1605,565,295,745,11330,11425,3235,3025,745,745
6,kensington cedar cottage,38411,32767,41800,1310,40485,6835,8905,7000,5395,...,2350,860,465,1020,15740,16655,4330,4615,1020,1020
7,kerrisdale,77248,57131,12065,615,11450,2410,1760,1390,1050,...,2115,360,290,1460,4170,3390,1315,2925,1460,1460
8,killarney,39013,33358,24675,1045,23635,4320,4925,3620,2995,...,1410,535,300,575,9245,9165,2975,2785,575,575
9,kitsilano,63092,49729,38450,900,37550,5240,5135,4600,4425,...,5940,1440,970,3530,10375,13060,6085,9465,3530,3530


In [20]:
# Step 1: Ensure "Neighbourhood" is the index in both DataFrames
df_1.set_index('Neighbourhood', inplace=True)  # Modify df_1 in place
hos_t.set_index('Neighbourhood', inplace=True)  # Modify hos_t in place
# df_1.set_index('Neighbourhood', inplace= False)
# hos_t.set_index('Neighbourhood', inplace= False)

# Step 2: Select the last 6 columns from df_1
last_6_columns_df1 = df_1.iloc[:, -6:]

# Step 3: Merge these columns into hos_t based on the "Neighbourhood" index
hos_t = hos_t.merge(last_6_columns_df1, how='left', left_index=True, right_index=True)

# Step 4: Reset the index if you want the "Neighbourhood" back as a column
hos_t.reset_index(inplace=True)

# Step 5: Display the updated hos_t DataFrame
(hos_t)


,Neighbourhood,Unnamed: 0,Street address,Zip/Postal Code,Latitude,Longitude,Region,Price,Bedrooms,Bathrooms,...,Total_moderate_crimes (2020-2024),Average_moderate_crimes (per_year),Total_minor_crimes (2020-2024),Average_minor_crimes (per_year),Class 6G,Class 5G,Class 4G,Class 3G,Class 2G,Class 1G
0,arbutus ridge,38,2795 W 37th Ave,V6N2T5,49.238765,-123.167839,Vancouver,5750000.0,5 bds,7 ba,...,322.0,64.0,1015.0,203.0,3185.0,2670.0,1090.0,2455.0,855.0,870.0
1,arbutus ridge,78,4422 Quesnel Dr,V6L2X6,49.247018,-123.168268,Vancouver,8680000.0,5 bds,6 ba,...,322.0,64.0,1015.0,203.0,3185.0,2670.0,1090.0,2455.0,855.0,870.0
2,arbutus ridge,125,4101 Yew St #304,V6L3B7,49.248583,-123.155246,Vancouver,699900.0,1 bd,1 ba,...,322.0,64.0,1015.0,203.0,3185.0,2670.0,1090.0,2455.0,855.0,870.0
3,arbutus ridge,175,2556 W 20th Ave,V6L1G8,49.254284,-123.163295,Vancouver,4290000.0,6 bds,5 ba,...,322.0,64.0,1015.0,203.0,3185.0,2670.0,1090.0,2455.0,855.0,870.0
4,arbutus ridge,184,2638 W 22nd Ave,V6L1M2,49.252494,-123.164150,Vancouver,5780000.0,4 bds,5 ba,...,322.0,64.0,1015.0,203.0,3185.0,2670.0,1090.0,2455.0,855.0,870.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1214,NaN,1093,2304 183 Keefer Place,NaN,54.505508,-128.585845,NaN,1295000.0,3 bd,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1215,NaN,1124,602 5629 Birney Avenue,NaN,49.253734,-123.234634,NaN,869000.0,1 bd,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1216,NaN,1149,205 5683 Hampton Place,NaN,49.184072,-122.945818,NaN,699999.0,1 bd,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1217,NaN,1151,310 5681 Birney Avenue,NaN,49.253734,-123.234634,NaN,898000.0,1 bd,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# Drop rows that have NaN in specific columns (e.g., 'Column1', 'Column2', ...)
final_df = hos_t.dropna(subset=['Class 6G'])

# Display the DataFrame after dropping rows
(final_df)


,Neighbourhood,Unnamed: 0,Street address,Zip/Postal Code,Latitude,Longitude,Region,Price,Bedrooms,Bathrooms,...,Total_moderate_crimes (2020-2024),Average_moderate_crimes (per_year),Total_minor_crimes (2020-2024),Average_minor_crimes (per_year),Class 6G,Class 5G,Class 4G,Class 3G,Class 2G,Class 1G
0,arbutus ridge,38,2795 W 37th Ave,V6N2T5,49.238765,-123.167839,Vancouver,5750000.0,5 bds,7 ba,...,322.0,64.0,1015.0,203.0,3185.0,2670.0,1090.0,2455.0,855.0,870.0
1,arbutus ridge,78,4422 Quesnel Dr,V6L2X6,49.247018,-123.168268,Vancouver,8680000.0,5 bds,6 ba,...,322.0,64.0,1015.0,203.0,3185.0,2670.0,1090.0,2455.0,855.0,870.0
2,arbutus ridge,125,4101 Yew St #304,V6L3B7,49.248583,-123.155246,Vancouver,699900.0,1 bd,1 ba,...,322.0,64.0,1015.0,203.0,3185.0,2670.0,1090.0,2455.0,855.0,870.0
3,arbutus ridge,175,2556 W 20th Ave,V6L1G8,49.254284,-123.163295,Vancouver,4290000.0,6 bds,5 ba,...,322.0,64.0,1015.0,203.0,3185.0,2670.0,1090.0,2455.0,855.0,870.0
4,arbutus ridge,184,2638 W 22nd Ave,V6L1M2,49.252494,-123.164150,Vancouver,5780000.0,4 bds,5 ba,...,322.0,64.0,1015.0,203.0,3185.0,2670.0,1090.0,2455.0,855.0,870.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,west point grey,934,4190 W 11th Avenue,NaN,49.262489,-123.199551,NaN,3088000.0,4 bd,3.0,...,353.0,71.0,1010.0,202.0,2355.0,2320.0,1055.0,2780.0,910.0,1165.0
1201,west point grey,952,4517 W 4th Avenue,NaN,49.269329,-123.210104,NaN,6999000.0,4 bd,4.0,...,353.0,71.0,1010.0,202.0,2355.0,2320.0,1055.0,2780.0,910.0,1165.0
1202,west point grey,1078,301 4375 W 10th Avenue,NaN,49.264023,-123.205245,NaN,799000.0,1 bd,1.0,...,353.0,71.0,1010.0,202.0,2355.0,2320.0,1055.0,2780.0,910.0,1165.0
1203,west point grey,1194,106 3731 W 6th Avenue,NaN,49.267160,-123.186370,NaN,585800.0,1 bd,1.0,...,353.0,71.0,1010.0,202.0,2355.0,2320.0,1055.0,2780.0,910.0,1165.0


In [22]:
# Step 1: Ensure "Neighbourhood" is the index in both DataFrames
df_2.set_index('Neighbourhood', inplace=True)  # Modify df_1 in place
hos_t.set_index('Neighbourhood', inplace=True)  # Modify hos_t in place
# df_1.set_index('Neighbourhood', inplace= False)
# hos_t.set_index('Neighbourhood', inplace= False)

# Step 2: Select the last 6 columns from df_1
last_6_columns_df2 = df_2.iloc[:, -6:]

# Step 3: Merge these columns into hos_t based on the "Neighbourhood" index
hos_t = hos_t.merge(last_6_columns_df2, how='left', left_index=True, right_index=True)

# Step 4: Reset the index if you want the "Neighbourhood" back as a column
hos_t.reset_index(inplace=True)

# Step 5: Display the updated hos_t DataFrame
(hos_t)


,Neighbourhood,Unnamed: 0,Street address,Zip/Postal Code,Latitude,Longitude,Region,Price,Bedrooms,Bathrooms,...,Class 4G,Class 3G,Class 2G,Class 1G,Class 6G_T,Class 5G_T,Class 4G_T,Class 3G_T,Class 2G_T,Class 1G_T
0,arbutus ridge,38,2795 W 37th Ave,V6N2T5,49.238765,-123.167839,Vancouver,5750000.0,5 bds,7 ba,...,1090.0,2455.0,855.0,870.0,4815.0,3750.0,1415.0,2575.0,1100.0,1100.0
1,arbutus ridge,78,4422 Quesnel Dr,V6L2X6,49.247018,-123.168268,Vancouver,8680000.0,5 bds,6 ba,...,1090.0,2455.0,855.0,870.0,4815.0,3750.0,1415.0,2575.0,1100.0,1100.0
2,arbutus ridge,125,4101 Yew St #304,V6L3B7,49.248583,-123.155246,Vancouver,699900.0,1 bd,1 ba,...,1090.0,2455.0,855.0,870.0,4815.0,3750.0,1415.0,2575.0,1100.0,1100.0
3,arbutus ridge,175,2556 W 20th Ave,V6L1G8,49.254284,-123.163295,Vancouver,4290000.0,6 bds,5 ba,...,1090.0,2455.0,855.0,870.0,4815.0,3750.0,1415.0,2575.0,1100.0,1100.0
4,arbutus ridge,184,2638 W 22nd Ave,V6L1M2,49.252494,-123.164150,Vancouver,5780000.0,4 bds,5 ba,...,1090.0,2455.0,855.0,870.0,4815.0,3750.0,1415.0,2575.0,1100.0,1100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1214,NaN,1093,2304 183 Keefer Place,NaN,54.505508,-128.585845,NaN,1295000.0,3 bd,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1215,NaN,1124,602 5629 Birney Avenue,NaN,49.253734,-123.234634,NaN,869000.0,1 bd,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1216,NaN,1149,205 5683 Hampton Place,NaN,49.184072,-122.945818,NaN,699999.0,1 bd,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1217,NaN,1151,310 5681 Birney Avenue,NaN,49.253734,-123.234634,NaN,898000.0,1 bd,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# Drop rows that have NaN in specific columns (e.g., 'Column1', 'Column2', ...)
final_df2 = hos_t.dropna(subset=['Class 6G_T'])

# Display the DataFrame after dropping rows
(final_df2)


,Neighbourhood,Unnamed: 0,Street address,Zip/Postal Code,Latitude,Longitude,Region,Price,Bedrooms,Bathrooms,...,Class 4G,Class 3G,Class 2G,Class 1G,Class 6G_T,Class 5G_T,Class 4G_T,Class 3G_T,Class 2G_T,Class 1G_T
0,arbutus ridge,38,2795 W 37th Ave,V6N2T5,49.238765,-123.167839,Vancouver,5750000.0,5 bds,7 ba,...,1090.0,2455.0,855.0,870.0,4815.0,3750.0,1415.0,2575.0,1100.0,1100.0
1,arbutus ridge,78,4422 Quesnel Dr,V6L2X6,49.247018,-123.168268,Vancouver,8680000.0,5 bds,6 ba,...,1090.0,2455.0,855.0,870.0,4815.0,3750.0,1415.0,2575.0,1100.0,1100.0
2,arbutus ridge,125,4101 Yew St #304,V6L3B7,49.248583,-123.155246,Vancouver,699900.0,1 bd,1 ba,...,1090.0,2455.0,855.0,870.0,4815.0,3750.0,1415.0,2575.0,1100.0,1100.0
3,arbutus ridge,175,2556 W 20th Ave,V6L1G8,49.254284,-123.163295,Vancouver,4290000.0,6 bds,5 ba,...,1090.0,2455.0,855.0,870.0,4815.0,3750.0,1415.0,2575.0,1100.0,1100.0
4,arbutus ridge,184,2638 W 22nd Ave,V6L1M2,49.252494,-123.164150,Vancouver,5780000.0,4 bds,5 ba,...,1090.0,2455.0,855.0,870.0,4815.0,3750.0,1415.0,2575.0,1100.0,1100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,west point grey,934,4190 W 11th Avenue,NaN,49.262489,-123.199551,NaN,3088000.0,4 bd,3.0,...,1055.0,2780.0,910.0,1165.0,3575.0,3045.0,1370.0,2880.0,1450.0,1450.0
1201,west point grey,952,4517 W 4th Avenue,NaN,49.269329,-123.210104,NaN,6999000.0,4 bd,4.0,...,1055.0,2780.0,910.0,1165.0,3575.0,3045.0,1370.0,2880.0,1450.0,1450.0
1202,west point grey,1078,301 4375 W 10th Avenue,NaN,49.264023,-123.205245,NaN,799000.0,1 bd,1.0,...,1055.0,2780.0,910.0,1165.0,3575.0,3045.0,1370.0,2880.0,1450.0,1450.0
1203,west point grey,1194,106 3731 W 6th Avenue,NaN,49.267160,-123.186370,NaN,585800.0,1 bd,1.0,...,1055.0,2780.0,910.0,1165.0,3575.0,3045.0,1370.0,2880.0,1450.0,1450.0


In [24]:
final_df2.to_csv('Vancouver_houses_with_crime_income_data_1.0km.csv', index=False)